<center>
<img src="./images/EAN.jpg" style="width:1200px">
</center>

<center>
<img src="./images/0_intro_ml.jpg" style="width:1200px">
</center>

## Lecture 5: Model Evaluation and Improvement
*Resampling methods, Grid Search, Evaluation Metrics and Scoring*


## Instructors:

>Leonardo A. Espinosa, PhD. Instructor.
(*email*: leonardo.espinosaleal@arcada.fi)

> Ruben D. Acosta, MSc. Instructor.
(*email*:  rdacostav@universidadean.edu.co)

# Learning goals for today
* Understand the different methods for validating data.
* Being able to understand the different metrics and its peculiarities.

1\. <a href="#/3/1">Resampling methods</a>

2\. <a href="#/14/1">Grid Search</a>:
   * <a href="#/15/1">Simple</a>.
   * <a href="#/24/1">With cross-validation</a>.
 
3\. <a href="#/41/1">Evaluation Metrics and Scoring</a>
   * <a href="#/42/1">Metrics for Binary Classification</a>.
   * <a href="#/78/1">Metrics for Multiclass Classification</a>.
   * <a href="#/83/1">Using Evaluation Metrics in Model Selection</a>
   
4\. <a href="#/88/1">Conclusions</a>

<center><img src="./images/data_squezz.jpg" style="width: 1450px;"/></center>

# Resampling methods

* Drawing samples from a training set and reffiting the model of interest on each sample. 
* A way to obtain additional information about the fitted model.

>Model assessment: Evaluating the model performance.

>Model selection: Selecting the proper level of flexibility.

# Cross-Validation
Cross-validation is primarily a way of measuring the predictive performance of a statistical model. 

### K-Fold cross-validation
<center><img src="./images/04-K-fold_cv.png" style="width: 750px;"/></center>

* Avoid *random* bias during selection of train and test data.
* How sensitive is our model to the selection of the training set.
* Computational cost is *k*.

### Stratified k-fold cross validation
<center><img src="./images/04-SK-fold_cv.png" alt="Drawing" style="width: 780px;"/></center>

* Keeps the proportion between classes in each fold as they are in the whole dataset.
* Good strategy with Imbalanced datasets (but use other strategies if proportion is more than 9 to 1).

### Leave-one-out cross-validation
<center><img src="./images/04-LOO_cv.png" alt="Drawing" style="width: 750px;"/></center>

* Good results in small datasets.
* Warning: Time consuming for large datasets.

### Shuffle-split cross-validation
<center><img src="./images/04-Random_cv.png" alt="Drawing" style="width: 750px;"/></center>
* Good strategy with large datasets.

# Bootstrapping

Quantify the uncertainty associated with a given estimator or statistical learning method.

* It is *easy* to derive estimates for standard errors and confidence intervals for complex estimators.
* It is not proven to work on finite samples. 

<center><img src="./images/bootstrap.png" alt="Drawing" style="width: 750px;"/></center>


<center>
<img src="./images/00_questions.jpg" style="width:1200px">
</center>

# Grid Search

Improve the models' generalization performance by tuning its parameters.


## Simple Grid Search

Using *for* loops over the parameters of a model.

In [ ]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import mglearn
from IPython.display import display

from matplotlib import rc
font = {'family' : 'monospace', 'weight' : 'bold', 'size'   : 25}
rc('font', **font) 

plt.rcParams['figure.figsize'] = [20, 10]
plt.rcParams['lines.linewidth'] = 5.0
plt.rcParams['lines.markersize'] = 15.0

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
warnings.filterwarnings("ignore", category=FutureWarning) 

In [ ]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

iris = load_iris()

X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, random_state=0)
print("Size of training set: {} size of test set: {}".format(X_train.shape[0], X_test.shape[0]))

In [ ]:
# naive grid search implementation
best_score = 0
for gamma in [0.001, 0.01, 0.1, 1, 10, 100]:
    for C in [0.001, 0.01, 0.1, 1, 10, 100]:
        # for each combination of parameters, train an SVC
        svm = SVC(gamma=gamma, C=C)
        svm.fit(X_train, y_train)
        # evaluate the SVC on the test set
        score = svm.score(X_test, y_test)
        # if we got a better score, store the score and parameters
        if score > best_score:
            best_score = score
            best_parameters = {'C': C, 'gamma': gamma}

print("Best score: {:.2f}".format(best_score))
print("Best parameters: {}".format(best_parameters))

## Overfiting *parameters* and the Validation Set

* We have tried different parameters and select the best ones on the *test set*.
* Test data should not be use to assess how good is the model.
* A new independent data set is necessary $\to$ Validation set.

<center><img src="./images/04-TVT_sets.png" alt="Drawing" style="width:1000px" align="middle"/></center>

* **Training set**: build the model.
* **Validation set**: select the parameters of the model.
* **Test set**: evaluate the performance of the selected parameters.

After selecting the best parameters using the validation set, we can rebuild a model using the parameter settings we found, but now training on both the training data and the validation data.

In [ ]:
from sklearn.svm import SVC

# split data into train+validation set and test set
X_trainval, X_test, y_trainval, y_test = train_test_split(iris.data, iris.target, random_state=0)

# split train+validation set into training and validation sets
X_train, X_valid, y_train, y_valid = train_test_split(X_trainval, y_trainval, random_state=1)

print("Size of training set: {} size of validation set: {} size of test set:" 
      " {}\n".format(X_train.shape[0], X_valid.shape[0], X_test.shape[0]))

In [ ]:
best_score = 0
for gamma in [0.001, 0.01, 0.1, 1, 10, 100]:
    for C in [0.001, 0.01, 0.1, 1, 10, 100]:
        # for each combination of parameters, train an SVC
        svm = SVC(gamma=gamma, C=C)
        svm.fit(X_train, y_train)
        # evaluate the SVC on the test set
        score = svm.score(X_valid, y_valid)
        # if we got a better score, store the score and parameters
        if score > best_score:
            best_score = score
            best_parameters = {'C': C, 'gamma': gamma}

In [ ]:
# rebuild a model on the combined training and validation set,
# and evaluate it on the test set
svm = SVC(**best_parameters)
svm.fit(X_trainval, y_trainval)
test_score = svm.score(X_test, y_test)
print("Best score on validation set: {:.2f}".format(best_score))
print("Best parameters: ", best_parameters)
print("Test set score with best parameters: {:.2f}".format(test_score))

## Grid Search with Cross-Validation

* For a better estimate of the generalization performance. 

* Instead of using a single split into a training and a validation set, we can use cross-validation to evaluate the performance of each parameter combination.

In [ ]:
from sklearn.model_selection import cross_val_score

for gamma in [0.001, 0.01, 0.1, 1, 10, 100]:
    for C in [0.001, 0.01, 0.1, 1, 10, 100]:
        # for each combination of parameters,
        # train an SVC
        svm = SVC(gamma=gamma, C=C)
        # perform cross-validation
        scores = cross_val_score(svm, X_trainval, y_trainval, cv=5)
        # compute mean cross-validation accuracy
        score = np.mean(scores)
        
        # if we got a better score, store the score and parameters
        if score > best_score:
            best_score = score
            best_parameters = {'C': C, 'gamma': gamma}

### rebuild a model on the combined training and validation set

In [ ]:

svm = SVC(**best_parameters)
svm.fit(X_trainval, y_trainval)

test_score = svm.score(X_test, y_test)
print("Best score on validation set: {:.2f}".format(best_score))
print("Best parameters: ", best_parameters)
print("Test set score with best parameters: {:.2f}".format(test_score))

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

param_grid = {'C': [0.001, 0.01, 0.1, 1, 10,120],
              'gamma': [0.001, 0.01, 0.1, 1, 10, 100]}

grid_search = GridSearchCV(SVC(), param_grid, cv=5)
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, random_state=0)
grid_search.fit(X_train, y_train)

print("Test set score: {:.2f}".format(grid_search.score(X_test, y_test)))
print("Best parameters: {}".format(grid_search.best_params_))
print("Best cross-validation score: {:.2f}\n".format(grid_search.best_score_))
print("Best estimator:\n{}".format(grid_search.best_estimator_))

**best$\underline{}$score$\underline{}$** $\to$ The  mean accuracy over the different splits for the parameters setting. On the training set.


### Analyzing the results (cv$\underline{}$results$\underline{}$)

We can inspect the results of the cross-validated grid search.

In [ ]:
# convert to DataFrame
results = pd.DataFrame(grid_search.cv_results_)
# show the first 5 rows
display(results.head())

In [ ]:
scores = np.array(results.mean_test_score).reshape(6, 6)
# plot the mean cross-validation scores
mglearn.tools.heatmap(scores, xlabel='gamma', xticklabels=param_grid['gamma'],
ylabel='C', yticklabels=param_grid['C'], cmap="viridis")

In [ ]:
param_grid_linear = {'C': np.linspace(1, 2, 6),
                     'gamma': np.linspace(1, 2, 6)}

param_grid_one_log = {'C': np.linspace(1, 2, 6),
                      'gamma': np.logspace(-3, 2, 6)}

param_grid_range = {'C': np.logspace(-3, 2, 6),
                    'gamma': np.logspace(-7, -2, 6)}

In [ ]:
svm?

In [ ]:
font = {'family' : 'monospace', 'weight' : 'bold', 'size'   : 18}
rc('font', **font) 
plt.rcParams['figure.figsize'] = [35, 10]

def plot_params():
    fig, axes = plt.subplots(1, 3)
    for param_grid, ax in zip([param_grid_linear, param_grid_one_log,
                               param_grid_range], axes):
        grid_search = GridSearchCV(SVC(), param_grid, cv=5)
        grid_search.fit(X_train, y_train)
        scores = grid_search.cv_results_['mean_test_score'].reshape(6, 6)

        # plot the mean cross-validation scores
        scores_image = mglearn.tools.heatmap(
                       scores, xlabel='gamma', ylabel='C', xticklabels=param_grid['gamma'],
                       yticklabels=param_grid['C'], cmap="viridis", ax=ax)
    plt.colorbar(scores_image, ax=axes.tolist())

In [ ]:
plot_params()

In [ ]:
font = {'family' : 'monospace', 'weight' : 'bold', 'size'   : 25}
rc('font', **font) 
plt.rcParams['figure.figsize'] = [25, 10]

### Search over spaces (not grids)

* In general is not a good idea.
* But, in some cases is possible to try *all possible* combinations of *all paramenters*.
* Pass to GridSearch a **list of dictionaries**. Each dictionary an independent grid.

In [ ]:
# We test the SVC classifier with two type of kernels.
param_grid = [{'kernel': ['rbf'],
               'C': [0.001, 0.01, 0.1, 1, 10, 100],
               'gamma': [0.001, 0.01, 0.1, 1, 10, 100]},
              {'kernel': ['linear'],
               'C': [0.001, 0.01, 0.1, 1, 10, 100]}]

grid_search = GridSearchCV(SVC(), param_grid, cv=5)
grid_search.fit(X_train, y_train)
print("Best parameters: {}".format(grid_search.best_params_))
print("Best cross-validation score: {:.2f}".format(grid_search.best_score_))

In [ ]:
results = pd.DataFrame(grid_search.cv_results_)
# we display the transposed table so that it better fits on the page:
display(results.T)

## Extra info


* You can Parallelize the cross-validation and grid search by passing the option `n_jobs=-1`

* GridSearchCV uses stratified *k*-fold cross-validation by default. But you can change it by passing a different
instruction to the **cv** variable. eg. 

    ```from sklearn.model_selection import LeaveOneOut
  loo = LeaveOneOut()
  grid_search = GridSearchCV(SVC(), param_grid, cv=loo)```



<center>
<img src="./images/00_questions.jpg" style="width:1200px">
</center>

<center>
<img src="./images/00_hands-on.jpg" style="width:1200px">
</center>

### Let's test gridsearch!

Here we run a gridsearch model for the dataset of your project:

* Create a grid of parameters searching for two hyperparamenters.
* Evaluate how the model improves. 

# Evaluation Metrics and Scoring

<center>
<img src="./images/metrics.png" style="width:1200px">
</center>

Default evaluation metrics in scikit-learn
* for classification: accuracy
* for regression: $R^2$

$R^2$ for regression is the usual one. It is a statistical measure of how close the data are to the fitted regression line. It is also known as the coefficient of determination, or the coefficient of multiple determination for multiple regression. However you can also use *Mean Absolute Error* or *Mean Squared Error* but the cases are limited. 

*When selecting a metric, you should always have the end goal of the machine learning application in mind.*

> business metric $\to$ business impact.

**be careful with imbalanced datasets**

In [ ]:
from sklearn.datasets import load_digits

digits = load_digits()
y = digits.target == 9
X_train, X_test, y_train, y_test = train_test_split(digits.data, y, random_state=0)

In [ ]:
from sklearn.dummy import DummyClassifier

dummy_majority = DummyClassifier(strategy='most_frequent').fit(X_train, y_train)
pred_most_frequent = dummy_majority.predict(X_test)
print("Unique predicted labels: {}".format(np.unique(pred_most_frequent)))
print("Test score: {:.2f}".format(dummy_majority.score(X_test, y_test)))

In [ ]:
from sklearn.tree import DecisionTreeClassifier

tree = DecisionTreeClassifier(max_depth=2).fit(X_train, y_train)
pred_tree = tree.predict(X_test)
print("Test score: {:.2f}".format(tree.score(X_test, y_test)))

In [ ]:
from sklearn.linear_model import LogisticRegression
dummy = DummyClassifier().fit(X_train, y_train)
pred_dummy = dummy.predict(X_test)
print("dummy score: {:.2f}".format(dummy.score(X_test, y_test)))

logreg = LogisticRegression(C=0.01).fit(X_train, y_train)
pred_logreg = logreg.predict(X_test)
print("logreg score: {:.2f}".format(logreg.score(X_test, y_test)))

## Confusion matrices

* rows $\to$ true classes
* columns $\to$ predicted classes

In [ ]:
from sklearn.metrics import confusion_matrix

confusion = confusion_matrix(y_test, pred_logreg)
print("Confusion matrix:\n{}".format(confusion))

In [ ]:
mglearn.plots.plot_confusion_matrix_illustration()

In [ ]:
font = {'family' : 'monospace', 'weight' : 'bold', 'size'   : 25}
rc('font', **font) 
plt.rcParams['figure.figsize'] = [25, 10]
mglearn.plots.plot_binary_confusion_matrix()

In [ ]:
print("Most frequent class:")
print(confusion_matrix(y_test,pred_most_frequent))
print("\nDummy model:")
print(confusion_matrix(y_test,pred_dummy))
print("\nDecision tree:")
print(confusion_matrix(y_test, pred_tree))
print("\nLogistic Regression")
print(confusion_matrix(y_test,pred_logreg))

## Accuracy: $\frac{TP+TN}{TP + TN + FP + FN}$

* Precision: $\frac{TP}{TP + FP}$ (fraction of relevant instances among the retrieved instances)

* Recall:    $\frac{TP}{TP + FN}$  (fraction of the total amount of relevant instances that were actually retrieved) 

* f-score:  $2\cdot \frac{precision\cdot recall}{precision+recall}$ (harmonic mean of precision and recall)

<center><img src="./images/precisionrecall.png" style="width: 1000px;"/></center>

<center><img src="./images/cm1.png" style="width: 1000px;"/></center>


<center><img src="./images/cm2.png" style="width: 800px;"/></center>


<center><img src="./images/cm1.png" style="width: 1000px;"/></center>


In [ ]:
from sklearn.metrics import f1_score

print("f1 score most frequent: {:.2f}".format(f1_score(y_test, pred_most_frequent)))
print("f1 score dummy: {:.2f}".format(f1_score(y_test, pred_dummy)))
print("f1 score tree: {:.2f}".format(f1_score(y_test, pred_tree)))
print("f1 score logistic regression: {:.2f}".format(f1_score(y_test, pred_logreg)))

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_test, pred_most_frequent,target_names=["not nine", "nine"]))

In [ ]:
print(classification_report(y_test, pred_dummy,target_names=["not nine", "nine"]))

In [ ]:
print(classification_report(y_test, pred_logreg,target_names=["not nine", "nine"]))

## Taking uncertainty into account

Most of classifiers provide a:

* decision_function
* predict_proba

You can modify the output by tuning a threshold.

>If you do set a threshold, don't use the test set. As with any other parameter, setting a decision threshold on the test set is likely to yield overly optimistic results. Use a validation set or cross-validation instead.

In [ ]:
from mglearn.datasets import make_blobs

X, y = make_blobs(n_samples=(400, 50), centers=2, cluster_std=[7.0, 2],random_state=22)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
svc = SVC(gamma=.1).fit(X_train, y_train)

In [ ]:
font = {'family' : 'monospace', 'weight' : 'bold', 'size'   : 12}
rc('font', **font) 
plt.rcParams['figure.figsize'] = [30, 10]

In [ ]:
mglearn.plots.plot_decision_threshold()

In [ ]:
print(classification_report(y_test, svc.predict(X_test)))

In [ ]:
y_pred_lower_threshold = svc.decision_function(X_test) > -.8
print(classification_report(y_test, y_pred_lower_threshold))

## Precision-recall curves and ROC curves

Adjust the trade-off of precision and recall for a given classifier.

In [ ]:
font = {'family' : 'monospace', 'weight' : 'bold', 'size'   : 25}
rc('font', **font) 
plt.rcParams['figure.figsize'] = [25, 15]
plt.rcParams['lines.markersize'] = 20
#plt.rcParams.keys()

In [ ]:
from sklearn.metrics import precision_recall_curve
from sklearn.ensemble import RandomForestClassifier

X, y = make_blobs(n_samples=(4000, 500), centers=2, cluster_std=[7.0, 2],
random_state=22)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
svc = SVC(gamma=.05).fit(X_train, y_train)
precision, recall, thresholds = precision_recall_curve(
y_test, svc.decision_function(X_test))
# find threshold closest to zero
close_zero = np.argmin(np.abs(thresholds))

rf = RandomForestClassifier(n_estimators=100, random_state=0, max_features=2)
rf.fit(X_train, y_train)
# RandomForestClassifier has predict_proba, but not decision_function
precision_rf, recall_rf, thresholds_rf = precision_recall_curve(
y_test, rf.predict_proba(X_test)[:, 1])

In [ ]:
def plot_pre_rec():
    plt.plot(precision, recall, label="svc")
    plt.plot(precision[close_zero], recall[close_zero], 'o',
             label="threshold zero svc", fillstyle="none", c='k', mew=2)
    plt.plot(precision_rf, recall_rf, label="rf")
    close_default_rf = np.argmin(np.abs(thresholds_rf - 0.5))
    plt.plot(precision_rf[close_default_rf], recall_rf[close_default_rf], '^', c='k', 
             label="threshold 0.5 rf", fillstyle="none", mew=2)
    plt.xlabel("Precision")
    plt.ylabel("Recall")
    plt.legend(loc="best")

In [ ]:
plot_pre_rec()

In [ ]:
from sklearn.metrics import average_precision_score
ap_rf = average_precision_score(y_test, rf.predict_proba(X_test)[:, 1])
ap_svc = average_precision_score(y_test, svc.decision_function(X_test))
print("Average precision of random forest: {:.3f}".format(ap_rf))
print("Average precision of svc: {:.3f}".format(ap_svc))

## Receiver operating characteristics (ROC) and AUC
* false positive rate (FPR) against the true positive rate (TPR)

<center><img src="./images/ROC_space.png" style="width: 850px;"/></center>


In [ ]:
from sklearn.metrics import roc_curve

def plot_roc_curve():
    fpr, tpr, thresholds = roc_curve(y_test, svc.decision_function(X_test))
    fpr_rf, tpr_rf, thresholds_rf = roc_curve(y_test, rf.predict_proba(X_test)[:, 1])

    close_zero = np.argmin(np.abs(thresholds))

    plt.plot(fpr, tpr, label="ROC Curve SVC")
    plt.plot(fpr_rf, tpr_rf, label="ROC Curve RF")
    plt.xlabel("FPR")
    plt.ylabel("TPR (recall)")
    plt.plot(fpr[close_zero], tpr[close_zero], 'o', 
    label="threshold zero SVC", fillstyle="none", c='k', mew=2)
    close_default_rf = np.argmin(np.abs(thresholds_rf - 0.5))
    plt.plot(fpr_rf[close_default_rf], tpr[close_default_rf], '^',
    label="threshold 0.5 RF", fillstyle="none", c='k', mew=2)
    plt.legend(loc=4)

In [ ]:
plot_roc_curve()

In [ ]:
from sklearn.metrics import roc_auc_score
rf_auc = roc_auc_score(y_test, rf.predict_proba(X_test)[:, 1])
svc_auc = roc_auc_score(y_test, svc.decision_function(X_test))
print("AUC for Random Forest: {:.3f}".format(rf_auc))
print("AUC for SVC: {:.3f}".format(svc_auc))

*AUC is a better metric for imbalanced classification problems than accuracy.*

## Metrics for Multiclass Classification

Same than binary classification metrics, but averaged over all classes.

In [ ]:
from sklearn.metrics import accuracy_score
X_train, X_test, y_train, y_test = train_test_split(
digits.data, digits.target, random_state=0)
lr = LogisticRegression().fit(X_train, y_train)
pred = lr.predict(X_test)
print("Accuracy: {:.3f}".format(accuracy_score(y_test, pred)))
print("Confusion matrix:\n{}".format(confusion_matrix(y_test, pred)))

In [ ]:
def plot_conf_matrix():
    scores_image = mglearn.tools.heatmap(
    confusion_matrix(y_test, pred), xlabel='Predicted label',
    ylabel='True label', xticklabels=digits.target_names,
    yticklabels=digits.target_names, cmap=plt.cm.gray_r, fmt="%d")
    plt.title("Confusion matrix")
    plt.gca().invert_yaxis()

In [ ]:
plot_conf_matrix()

In [ ]:
print(classification_report(y_test, pred))

## Using Evaluation Metrics in Model Selection

How to include your metric in your model selection using GridSearchCV or cross_val_score.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(digits.data, digits.target == 9, random_state=0)
# we provide a somewhat bad grid to illustrate the point:
param_grid = {'gamma': [0.0001, 0.01, 0.1, 1, 10]}
# using the default scoring of accuracy:
grid = GridSearchCV(SVC(), param_grid=param_grid)
grid.fit(X_train, y_train)
print("Grid-Search with accuracy")
print("Best parameters:", grid.best_params_)
print("Best cross-validation score (accuracy)): {:.3f}".format(grid.best_score_))
print("Test set AUC: {:.3f}".format(
roc_auc_score(y_test, grid.decision_function(X_test))))
print("Test set accuracy: {:.3f}".format(grid.score(X_test, y_test)))

In [ ]:
# using AUC scoring instead:
grid = GridSearchCV(SVC(), param_grid=param_grid, scoring="roc_auc")
grid.fit(X_train, y_train)
print("\nGrid-Search with AUC")
print("Best parameters:", grid.best_params_)
print("Best cross-validation score (AUC): {:.3f}".format(grid.best_score_))
print("Test set AUC: {:.3f}".format(
roc_auc_score(y_test, grid.decision_function(X_test))))
print("Test set accuracy: {:.3f}".format(grid.score(X_test, y_test)))

In [ ]:
from sklearn.metrics.scorer import SCORERS
print("Available scorers:\n{}".format(sorted(SCORERS.keys())))

<center>
<img src="./images/00_questions.jpg" style="width:1200px">
</center>

<center>
<img src="./images/00_hands-on.jpg" style="width:1200px">
</center>

# Exercise

Here we are going to model the *identification glass dataset*
https://archive.ics.uci.edu/ml/datasets/glass+identification

1. Explore the Dataset
2. Fit a same model using k-fold cross-validation and RepeatedStratifiedKFold.
3. Compare different scoring methods: accuracy and auc.
4. What is the final result? Improves?

# Conclusions

* Cross-validation or the use of a test set allow us to evaluate a machine learning model as it will perform in the future.
* Make sure that the metric you choose to evaluate and select a model for is a good stand-in for what the model will actually be used for.

<center>
<img src="./images/00_thats_all.jpg" style="width:1200px">
</center>